<a href="https://colab.research.google.com/github/deb-kit2/DeepDream/blob/master/DeepDream.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###Dependencies

In [0]:
from keras.applications import inception_v3
from keras import backend as k

k.set_learning_phase(0)

import numpy as np
import scipy
from keras.preprocessing.image import load_img, save_img, img_to_array

###hello


In [0]:
some_layers= {
    "mixed2" : 2.,
    "mixed3" : 0.5,
    "mixed4" : 1.,
    "mixed5" : 0.2, 
}

In [0]:
model = inception_v3.InceptionV3(include_top = False, weights = "imagenet")

loss = K.variable(0.)

for layer, coeff in some_layers.items():
  activation = model.get_layer(layer).output
  loss += coeff * K.sum(K.square(activation))

In [0]:
my_image = model.input

